In [1]:
import clip
import numpy as np
import torch
from pkg_resources import packaging
from PIL import Image
import os
import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
import sys
sys.path.insert(0, '../')
from fair_pca import FairPCA,FairKernelPCA
from sklearn.decomposition import PCA
import pandas as pd
from __future__ import print_function, division
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import utils as ut
plt.ion()

import miap_dataset as md
import importlib
from sklearn.metrics import accuracy_score

In [2]:
importlib.reload(ut)
importlib.reload(md)


<module 'miap_dataset' from '/mnt/efs/fairclip/FinalCode/MIAP/miap_dataset.py'>

In [3]:
device = "cuda:2" #if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [4]:
miap_dataset_orig = md.MIAPDataset('../../miap','test' )
miap_dataset_trans = md.MIAPDataset('../../miap', 'test',transform = preprocess)

In [5]:
len(miap_dataset_orig)

22590

In [6]:
def get_features(dataset, model):
    all_features = []
    all_gender = []
    all_age = []
    all_area = []
    all_num_people = []
    with torch.no_grad():
        for images, gen, age, area, num_people in tqdm(DataLoader(dataset, batch_size=100)):
            
            features = model.encode_image(images.to(device))
            all_features.append(features)
            all_gender.append(gen)
            all_age.append(age)
            all_area.append(area)
            all_num_people.append(num_people)
    return torch.cat(all_features), torch.cat(all_gender).cpu().numpy(), torch.cat(all_area).cpu().numpy(), torch.cat(all_age).cpu().numpy(), torch.cat(all_num_people).cpu().numpy()

In [7]:
features, gender, area, age, no_people =  get_features(miap_dataset_trans, model)
features /= features.norm(dim=-1, keepdim=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [56:40<00:00, 15.05s/it]


In [8]:
classification_tasks = {}
classification_tasks['gender'] = [
#     ('a photo of a woman','a photo of a man'),
    ('an inconspicuous photo of a person','a prominent photo of a person'),
    ('a photo of a child', 'a photo of an adult'),
    ('a photo of one person', 'a photo of more than one person'),
    
]
gt_labels = {0:area,1:age,2:no_people }

In [9]:
def calc_eop(text_features, image_features,gt_labels, attr, method, gender):
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1).cpu().numpy().astype(np.float64)
    predictions = np.argmax(similarity,axis=1)
    idx_ = np.where(np.logical_and(gender != 2, age != 2))
    accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
    print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]) )

    for ell in range(2):
        idx_ = np.where(np.logical_and(gender == ell, age != 2))            
        idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]

        pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
        temp[cc, ell] = idx_tp.shape[0]/ pos_
            

    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    temp.to_csv(f"../results_csv/{attr}_miap_clf_{method}.csv")
    print(temp)

In [10]:
for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs = []
    tp_idxs = []
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(word) for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * features @ text_features.T).softmax(dim=-1).cpu().numpy().astype(np.float64)
        predictions = np.argmax(similarity,axis=1)
        idx_ = np.where(np.logical_and(gender != 2, age != 2))
        accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
        print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]) )
#         if cc == 0:
#             continue
        
        for ell in range(2):
#             print(ell)
            idx_ = np.where(np.logical_and(gender == ell, age != 2))
#             print(idx_[0].shape)
#             temp[cc, ell] = np.around(np.mean(predictions[id_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
            idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
#             temp[cc, ell] = 1 - np.around(np.mean(predictions[data['labels']['gender']== ell] != data['labels'][]),2)
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    temp.to_csv(f"../results_csv/{attr}_miap_clf_orig.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('an inconspicuous photo of a person', 'a prominent photo of a person') 0.62 0.8427157001414427
('a photo of a child', 'a photo of an adult') 0.92 0.9181046676096181
('a photo of one person', 'a photo of more than one person') 0.66 0.2780763790664781
                                                      Female      Male  \
(an inconspicuous photo of a person, a prominen...  0.925672  0.923591   
(a photo of a child, a photo of an adult)           0.867606  0.945474   
(a photo of one person, a photo of more than on...  0.218419  0.652610   

                                                    Disparity  Accuracy  
(an inconspicuous photo of a person, a prominen...   0.002081      0.62  
(a photo of a child, a photo of an adult)            0.077869      0.92  
(a photo of one person, a photo of more than on...   0

In [11]:
# --- Evaluation of zero-shot classification w.r.t. gender  -------------------------
# Numbers are the mean prediction rate for the first word when classifying into the two words
# ('an inconspicuous photo of a person', 'a prominent photo of a person') 0.62 0.8407355021216407
# ('a photo of a child', 'a photo of an adult') 0.92 0.9183875530410184
# ('a photo of one person', 'a photo of more than one person') 0.66 0.2775106082036775
#                                                       Female      Male  \
# (an inconspicuous photo of a person, a prominen...  0.921670  0.922159   
# (a photo of a child, a photo of an adult)           0.867606  0.945911   
# (a photo of one person, a photo of more than on...  0.216553  0.653949   

#                                                     Disparity  Accuracy  
# (an inconspicuous photo of a person, a prominen...   0.000489      0.62  
# (a photo of a child, a photo of an adult)            0.078305      0.92  
# (a photo of one person, a photo of more than on...   0.437397      0.66 

SyntaxError: invalid syntax (368994024.py, line 1)

In [12]:
projection_GT,projection_inferred, MI_GT, MI_inferred = ut.calculate_projections_miap(model, preprocess, device)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 700/700 [3:29:43<00:00, 17.98s/it]


Error of predicting gender train = 0.05


In [13]:
# FPCA GT
from scipy.special import softmax

for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs = []
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_GT[attr]
        idx_ = np.where(np.logical_and(gender != 2, age != 2))
        all_features_val_transf = projection_train.just_transform(features.cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]))
        accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
#         if cc == 0:
#             continue
        for ell in range(2):
            idx_ = np.where(np.logical_and(gender == ell, age != 2))
            
#             temp[cc, ell] = np.around(np.mean(predictions[idx_] == data['labels'][gt_label[cc]][idx_]),2)
            
            
            idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_miap_clf_fpca_gt.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('an inconspicuous photo of a person', 'a prominent photo of a person') 0.6 0.2079207920792079
('a photo of a child', 'a photo of an adult') 0.97 0.9680339462517681
('a photo of one person', 'a photo of more than one person') 0.62 0.3968882602545969
                                                      Female      Male  \
(an inconspicuous photo of a person, a prominen...  0.300743  0.334288   
(a photo of a child, a photo of an adult)           0.956539  0.974264   
(a photo of one person, a photo of more than on...  0.387679  0.655957   

                                                    Disparity  Accuracy  
(an inconspicuous photo of a person, a prominen...   0.033545      0.60  
(a photo of a child, a photo of an adult)            0.017725      0.97  
(a photo of one person, a photo of more than on...   0.

In [ ]:
# Numbers are the mean prediction rate for the first word when classifying into the two words
# ('an inconspicuous photo of a person', 'a prominent photo of a person') 0.59 0.20862800565770862
# ('a photo of a child', 'a photo of an adult') 0.97 0.9677510608203678
# ('a photo of one person', 'a photo of more than one person') 0.62 0.3983026874115983
#                                                       Female      Male  \
# (an inconspicuous photo of a person, a prominen...  0.301887  0.333333   
# (a photo of a child, a photo of an adult)           0.956942  0.973610   
# (a photo of one person, a photo of more than on...  0.389546  0.657965   

#                                                     Disparity  Accuracy  
# (an inconspicuous photo of a person, a prominen...   0.031447      0.59  
# (a photo of a child, a photo of an adult)            0.016668      0.97  
# (a photo of one person, a photo of more than on...   0.268419      0.62

In [14]:
# FPCA inf
from scipy.special import softmax

for attr in ['gender']:
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    temp = np.zeros((len(classification_tasks[attr]),2))
    accs = []
    for cc, task in enumerate(classification_tasks[attr]):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        projection_train = projection_inferred[attr]
        idx_ = np.where(np.logical_and(gender != 2, age != 2))
        all_features_val_transf = projection_train.just_transform(features.cpu().numpy().astype(np.float64))
        text_features_pca = projection_train.just_transform(text_features.cpu().numpy().astype(np.float64))
        similarity = softmax(100.0 * np.matmul(all_features_val_transf, np.transpose(text_features_pca)),axis=1)
        
        predictions = np.argmax(similarity,axis=1)
        print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]))
        accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
#         if cc == 0:
#             continue
        for ell in range(2):
            idx_ = np.where(np.logical_and(gender == ell, age != 2))
           
            
            idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]
            
            pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
#             print("pos: ", pos_)
            temp[cc, ell] = idx_tp.shape[0]/ pos_
            
    columns= ['Female', 'Male']
    temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
    if attr == 'gender':	  
        temp['Disparity'] = abs(temp['Male'] - temp['Female'])
        temp['Accuracy'] = accs
    elif attr == 'race':
        temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
    temp.to_csv(f"../results_csv/{attr}_miap_clf_fpca_inf.csv")
    print(temp)

--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
('an inconspicuous photo of a person', 'a prominent photo of a person') 0.6 0.20947666195190948
('a photo of a child', 'a photo of an adult') 0.97 0.9694483734087694
('a photo of one person', 'a photo of more than one person') 0.61 0.3908062234794908
                                                      Female      Male  \
(an inconspicuous photo of a person, a prominen...  0.308748  0.333811   
(a photo of a child, a photo of an adult)           0.960563  0.974264   
(a photo of one person, a photo of more than on...  0.359676  0.664659   

                                                    Disparity  Accuracy  
(an inconspicuous photo of a person, a prominen...   0.025063      0.60  
(a photo of a child, a photo of an adult)            0.013701      0.97  
(a photo of one person, a photo of more than on...   0

In [16]:
print("======== Running CLF MI G.T on the model ============== ")

for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_GT[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
        accs = []
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            idx_ = np.where(np.logical_and(gender != 2, age != 2))
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)

            predictions = np.argmax(similarity,axis=1)
            print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]))
            accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
#             if cc == 0:
#                 continue
            for ell in range(2):
                idx_ = np.where(np.logical_and(gender == ell, age != 2))
           
            
                idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]

                pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
#                 print(idx_[0].shape, pos_)
                temp[cc, ell] = idx_tp.shape[0]/ pos_
                
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender':	  
            temp['Disparity'] = abs(temp['Male'] - temp['Female'])
            temp['Accuracy'] = accs
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_miap_clf_MI_gt{num_clip}.csv")
        print(temp)
        print(np.mean(temp.Disparity))

======== Running CLF MI G.T on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
('an inconspicuous photo of a person', 'a prominent photo of a person') 0.6 0.7015558698727016
('a photo of a child', 'a photo of an adult') 0.66 0.6598302687411598
('a photo of one person', 'a photo of more than one person') 0.54 0.3760961810466761
                                                      Female      Male  \
(an inconspicuous photo of a person, a prominen...  0.762150  0.785100   
(a photo of a child, a photo of an adult)           0.678873  0.649509   
(a photo of one person, a photo of more than on...  0.357187  0.459839   

                                                    Disparity  Accuracy  
(an inconspicuous photo of a person, a prominen...   0.022950      0.60  
(a photo of a child, a photo of an adult) 

In [15]:
print("======== Running CLF MI inferred on the model ============== ")

for attr in ['gender']:
    num_clip_s = [400, 256]
    mis = MI_inferred[attr]
    print(f'--- Evaluation of zero-shot classification w.r.t. {attr}  -------------------------')
    print('Numbers are the mean prediction rate for the first word when classifying into the two words')
    for num_clip in num_clip_s:
        print(f"----------- {num_clip}--------------")
        temp = np.zeros((len(classification_tasks[attr]),2))
        accs = []
        for cc, task in enumerate(classification_tasks[attr]):
            text_inputs = torch.cat([clip.tokenize(f"a photo of a person {word}.") for word in task]).to(device)
            with torch.no_grad():
                text_features = model.encode_text(text_inputs)
            text_features_mi =text_features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            image_features_val = features.cpu().numpy().astype(np.float64)[:, mis[:num_clip]]
            similarity = softmax(100.0 * np.matmul(image_features_val, np.transpose(text_features_mi)),axis=1)
            idx_ = np.where(np.logical_and(gender != 2, age != 2))
            predictions = np.argmax(similarity,axis=1)
            print(task,np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2), np.mean(predictions[idx_]))
            accs.append(np.around(np.mean(predictions[idx_] == gt_labels[cc][idx_]),2))
#             if cc == 0:
#                 continue
            for ell in range(2):
                idx_ = np.where(np.logical_and(gender == ell, age != 2))
           
            
                idx_tp = np.where(np.logical_and(gt_labels[cc][idx_] == 1, predictions[idx_] == 1))[0]

                pos_ = np.where(gt_labels[cc][idx_] == 1)[0].shape[0]
#                 print(idx_[0].shape, pos_)
                temp[cc, ell] = idx_tp.shape[0]/ pos_
                
        columns= ['Female', 'Male']
        temp = pd.DataFrame(temp, columns=columns, index=classification_tasks[attr])
        if attr == 'gender':	  
            temp['Disparity'] = abs(temp['Male'] - temp['Female'])
            temp['Accuracy'] = accs
        elif attr == 'race':
            temp['Disparity'] = temp.max(axis = 1) - temp.min(axis = 1)
        temp.to_csv(f"../results_csv/{attr}_miap_clf_MI_inf{num_clip}.csv")
        print(temp)
        print(np.mean(temp.Disparity))

======== Running CLF MI inferred on the model ============== 
--- Evaluation of zero-shot classification w.r.t. gender  -------------------------
Numbers are the mean prediction rate for the first word when classifying into the two words
----------- 400--------------
('an inconspicuous photo of a person', 'a prominent photo of a person') 0.63 0.660961810466761
('a photo of a child', 'a photo of an adult') 0.67 0.6684582743988685
('a photo of one person', 'a photo of more than one person') 0.55 0.27934936350777934
                                                      Female      Male  \
(an inconspicuous photo of a person, a prominen...  0.759291  0.768386   
(a photo of a child, a photo of an adult)           0.681288  0.661505   
(a photo of one person, a photo of more than on...  0.215308  0.394913   

                                                    Disparity  Accuracy  
(an inconspicuous photo of a person, a prominen...   0.009095      0.63  
(a photo of a child, a photo of an a